# Write Output to CSV
This submodule implements the writing of all input data (and eventually results, but not yet)
to CSV files.

In [1]:
from __future__ import print_function

In [2]:
from salib import extend, import_notebooks
from Tables import Table

In [3]:
from Frame2D_Base import Frame2D
import Frame2D_Input

In [4]:
#test:
f = Frame2D('frame-1')
f.input_all()

## General Table Writing
Define a general method to write all tables.  By default, a table of a given name
will be used from the '`self.rawdata`' object.  If table '`xxx`' does not exist
there, it will be created by calling method '`.list_xxx()`' to generate the data,
and formed by using the list of columns in class attribute '`.COLUMNS_xxx`'.

`.list_xxx()` must return a list of tuples or dictionaries, each of these being one
row of the table.  If a tuple, that data is obviously given in the same order as the columns.
If a dictionary, the keys are the column names (see 'Member Loads', below).

In [5]:
@extend
class Frame2D:
    
    def write_table(self,table_name,ds_name=None,prefix=None,record=True,precision=None,args=(),makedir=False):
        t = getattr(self.rawdata,table_name,None)
        if t is None:
            methodname = 'list_'+table_name
            method = getattr(self,methodname,None)
            if method and callable(method):
                data = method(*args)
                t = Table(table_name,data=data,columns=getattr(self,'COLUMNS_'+table_name))
        if t is None:
            raise ValueError("Unable to find table '{}'".format(table_name))
        t.write(ds_name=ds_name,prefix=prefix,precision=precision,makedir=makedir)
        if record:
            setattr(self.rawdata,table_name,t)
        return t            

In [6]:
##test:
try:
    f.write_table('foobarzx3')
except Exception, e:
    print('**** Error:',e)

**** Error: Unable to find table 'foobarzx3'


### Nodes

In [7]:
 @extend
class Frame2D:
    
    def list_nodes(self):
        return [(n.id,n.x,n.y) for n in self.nodes.values()]

In [8]:
##test:
t = f.write_table('nodes','test-1')
t.file_name

'test-1.d/nodes.csv'

In [9]:
##test:
%cat test-1.d/nodes.csv

NODEID,X,Y
A,0.0,0.0
B,0.0,4000.0
C,8000.0,4000.0
D,8000.0,0.0


Delete the `nodes` table from `.rawdata` so that we can test that `.list_nodes()` is properly called
to regenerated the table.

In [10]:
##test:
if hasattr(f.rawdata,'nodes'):
    delattr(f.rawdata,'nodes')
%rm test-1.d/nodes.csv
t = f.write_table('nodes','test-1')
t.file_name

'test-1.d/nodes.csv'

In [11]:
##test:
%cat test-1.d/nodes.csv

NODEID,X,Y
A,0.0,0.0
B,0.0,4000.0
C,8000.0,4000.0
D,8000.0,0.0


### Supports

In [12]:
@extend
class Frame2D:
    
    def list_supports(self):
        ans = []
        for node in self.nodes.values():
            if node.constraints:
                cl = tuple(node.constraints)
                if len(cl) < 3:
                    cl = cl + ('',)*(3-len(cl))
                ans.append((node.id,)+cl)
        return ans

In [13]:
##test:
f.list_supports()

[('A', 'FX', 'FY', 'MZ'), ('D', 'FX', 'FY', '')]

In [14]:
##test:
t = f.write_table('supports','test-1')
t.file_name

'test-1.d/supports.csv'

In [15]:
##test:
%cat test-1.d/supports.csv

NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY,


In [16]:
##test:
x = 'supports'
if hasattr(f.rawdata,x):
    delattr(f.rawdata,x)
%rm test-1.d/supports.csv
t = f.write_table(x,'test-1')
t.file_name

'test-1.d/supports.csv'

In [17]:
##test:
%cat test-1.d/supports.csv

NODEID,C0,C1,C2
A,FX,FY,MZ
D,FX,FY,


### Members

In [18]:
@extend
class Frame2D:
    
    def list_members(self):
        return [(m.id,m.nodej.id,m.nodek.id) for m in self.members.values()]

In [19]:
##test:
f.list_members()

[('AB', 'A', 'B'), ('BC', 'B', 'C'), ('CD', 'C', 'D')]

In [20]:
##test:
f.write_table('members','test-1')
%cat test-1.d/members.csv

MEMBERID,NODEJ,NODEK
AB,A,B
BC,B,C
CD,C,D


### Releases

In [21]:
@extend
class Frame2D:
    
    def list_releases(self):
        return [(m.id,)+tuple(m.releases) for m in self.members.values() if m.releases]

In [22]:
##test:
f.list_releases()

[('AB', 'MZK'), ('CD', 'MZJ')]

In [23]:
##test:
f.write_table('releases','test-1')
%cat test-1.d/releases.csv

MEMBERID,RELEASE
AB,MZK
CD,MZJ


### Properties

In [24]:
@extend
class Frame2D:
    
    def list_properties(self):
        return [(m.id,m.size,m.Ix,m.A) for m in self.members.values()]

In [25]:
##test:
f.list_properties()

[('AB', 'W310x97', 222000000.0, 12300.0),
 ('BC', 'W460x106', 488000000.0, 13500.0),
 ('CD', '', 222000000.0, 12300.0)]

In [26]:
##test:
f.write_table('properties','test-1')
%cat test-1.d/properties.csv

MEMBERID,SIZE,IX,A
BC,W460x106,488000000.0,13500.0
AB,W310x97,222000000.0,12300.0
CD,,222000000.0,12300.0


### Node Loads

In [27]:
@extend
class Frame2D:
    
    def list_node_loads(self):
        ans = []
        dirns = ['FX','FY','FZ']
        for loadid,node,nload in self.nodeloads:
            for i in [0,1,2]:
                if nload[i]:
                    ans.append((loadid,node.id,dirns[i],nload[i]))
        return ans

In [28]:
##test:
f.list_node_loads()

[('wind', 'B', 'FX', -200000.0)]

In [29]:
##test:
f.write_table('node_loads','test-1')
%cat test-1.d/node_loads.csv

LOAD,NODEID,DIRN,F
Wind,B,FX,-200000.0


### Support Displacements

In [30]:
@extend    
class Frame2D:
    
    def list_support_displacements(self):
        ans = []
        dirns = ['DX','DY','RZ']
        for loadid,node,nload in self.nodedeltas:
            for i in [0,1,2]:
                if nload[i]:
                    ans.append((loadid,node.id,dirns[i],nload[i]))
        return ans        

In [31]:
##test:
f.list_support_displacements()

[('other', 'A', 'DY', -10.0)]

In [32]:
##test:
f.write_table('support_displacements','test-1')
%cat test-1.d/support_displacements.csv

LOAD,NODEID,DIRN,DELTA
Other,A,DY,-10


### Member Loads

In [33]:
from MemberLoads import unmakeMemberLoad

@extend
class Frame2D:
    
    def list_member_loads(self):
        ans = []
        for loadid,memb,mload in self.memberloads:
            ml = unmakeMemberLoad(mload)
            ml['MEMBERID'] = memb.id
            ml['LOAD'] = loadid
            ans.append(ml)
        return ans

In [34]:
##test:
f.list_member_loads()

[{'LOAD': 'live', 'MEMBERID': 'BC', 'TYPE': 'UDL', 'W1': -50},
 {'A': 5000.0, 'LOAD': 'live', 'MEMBERID': 'BC', 'TYPE': 'PL', 'W1': -200000}]

In [35]:
##test:
f.write_table('member_loads','test-1')
%cat test-1.d/member_loads.csv

LOAD,MEMBERID,TYPE,W1,W2,A,B,C
Live,BC,UDL,-50,,,,
Live,BC,PL,-200000,,5000.0,,


In [36]:
##test:
if hasattr(f.rawdata,'member_loads'):
    delattr(f.rawdata,'member_loads')
%rm test-1.d/member_loads.csv
f.write_table('member_loads','test-1')
%cat test-1.d/member_loads.csv

LOAD,MEMBERID,TYPE,W1,W2,A,B,C
live,BC,UDL,-50,,,,
live,BC,PL,-200000,,5000.0,,


### Load Combinations

In [37]:
@extend
class Frame2D:
    
    def list_load_combinations(self):
        return [(case,load,factor) for case,load,factor in self.loadcombinations]

In [38]:
##test:
f.list_load_combinations()

[('one', 'live', 1.5),
 ('one', 'wind', 1.75),
 ('all', 'live', 1.0),
 ('all', 'other', 1.0),
 ('all', 'wind', 1.0)]

In [39]:
##test:
f.write_table('load_combinations','test-1')
%cat test-1.d/load_combinations.csv

CASE,LOAD,FACTOR
One,Live,1.5
One,Wind,1.75


### File Signatures

In [40]:
##test:
vars(f.rawdata).keys()

['member_loads',
 'releases',
 'supports',
 'support_displacements',
 'load_combinations',
 'members',
 'nodes',
 'node_loads',
 'properties']

In [41]:
@extend
class Frame2D:
    
    COLUMNS_signatures = ['NAME','PATH','SIGNATURE']
    
    def list_signatures(self):
        return [t.signature() for tn,t in vars(self.rawdata).items() if type(t) is Table]

In [42]:
##test:
f.list_signatures()

[('member_loads',
  'test-1.d/member_loads.csv',
  'a9fcdf2e2ae939fe6506ad063f92540bb9816856b51481df6be0cc2657467aa0'),
 ('releases',
  'test-1.d/releases.csv',
  'ae3d4681f60b6c5665cfd07a4894a01dd386cc7a51e98eb08248fc202e41e09f'),
 ('supports',
  'test-1.d/supports.csv',
  'c9b14f3fb0868a679912d6d5d8f23e75f89cc8accd8e9f2b1613bc9710653815'),
 ('support_displacements',
  'test-1.d/support_displacements.csv',
  'b475cca77a5ddcf815425514c484fcb3506195db840a9bf3006a187c9e457617'),
 ('load_combinations',
  'test-1.d/load_combinations.csv',
  'efe64703cadacf5f169f913b9d49dcf4c6523dd62e474cfff90782787c830310'),
 ('members',
  'test-1.d/members.csv',
  '783cee2e924a1f1fc738638af431671f02e25f1b97430d834a226eecd3dd09e3'),
 ('nodes',
  'test-1.d/nodes.csv',
  'a1c6202ec4a3f745cbde4e96c7b07ce173d14ebaeaa6956b40260053ffb34e6c'),
 ('node_loads',
  'test-1.d/node_loads.csv',
  'c6283d540ec54fe3f9a92ac7f0ec98e74f3e8351dc77387e3d113bc5e618ced4'),
 ('properties',
  'test-1.d/properties.csv',
  '4ebc2477

In [43]:
##test:
f.write_table('signatures','test-1',record=False)
%cat test-1.d/signatures.csv

NAME,PATH,SIGNATURE
member_loads,test-1.d/member_loads.csv,a9fcdf2e2ae939fe6506ad063f92540bb9816856b51481df6be0cc2657467aa0
releases,test-1.d/releases.csv,ae3d4681f60b6c5665cfd07a4894a01dd386cc7a51e98eb08248fc202e41e09f
supports,test-1.d/supports.csv,c9b14f3fb0868a679912d6d5d8f23e75f89cc8accd8e9f2b1613bc9710653815
support_displacements,test-1.d/support_displacements.csv,b475cca77a5ddcf815425514c484fcb3506195db840a9bf3006a187c9e457617
load_combinations,test-1.d/load_combinations.csv,efe64703cadacf5f169f913b9d49dcf4c6523dd62e474cfff90782787c830310
members,test-1.d/members.csv,783cee2e924a1f1fc738638af431671f02e25f1b97430d834a226eecd3dd09e3
nodes,test-1.d/nodes.csv,a1c6202ec4a3f745cbde4e96c7b07ce173d14ebaeaa6956b40260053ffb34e6c
node_loads,test-1.d/node_loads.csv,c6283d540ec54fe3f9a92ac7f0ec98e74f3e8351dc77387e3d113bc5e618ced4
properties,test-1.d/properties.csv,4ebc2477cadb207459034659d0bf1da9b9dbb02bb06af2c060894ecb595effca


## Write everything

In [44]:
import os, os.path

@extend
class Frame2D:
    
    def write_all(self,ds_name,mkdir=False):
        if mkdir:
            dname = ds_name + '.d'
            if not os.path.exists(dname):
                os.mkdir(dname)
        self.write_table('nodes',ds_name)
        self.write_table('supports',ds_name)
        self.write_table('members',ds_name)
        self.write_table('releases',ds_name)
        self.write_table('properties',ds_name)
        self.write_table('node_loads',ds_name)
        self.write_table('support_displacements',ds_name)
        self.write_table('member_loads',ds_name)
        self.write_table('load_combinations',ds_name)
        self.write_table('signatures',ds_name,record=False)

In [45]:
##test:
%rm test-1.d/*
%ls -l test-1.d
f.reset()
f.input_all()
f.write_all('test-1')
%ls -l test-1.d

rm: cannot remove ‘test-1.d/all’: Is a directory
rm: cannot remove ‘test-1.d/One’: Is a directory
total 8
drwxr-xr-x 2 nholtz nholtz 4096 Jun  3 18:02 all/
drwxr-xr-x 2 nholtz nholtz 4096 Jun  3 18:08 One/
total 48
drwxr-xr-x 2 nholtz nholtz 4096 Jun  3 18:02 all/
-rw-r--r-- 1 nholtz nholtz   44 Jun  3 18:10 load_combinations.csv
-rw-r--r-- 1 nholtz nholtz   80 Jun  3 18:10 member_loads.csv
-rw-r--r-- 1 nholtz nholtz   42 Jun  3 18:10 members.csv
-rw-r--r-- 1 nholtz nholtz   39 Jun  3 18:10 node_loads.csv
-rw-r--r-- 1 nholtz nholtz   63 Jun  3 18:10 nodes.csv
drwxr-xr-x 2 nholtz nholtz 4096 Jun  3 18:08 One/
-rw-r--r-- 1 nholtz nholtz  106 Jun  3 18:10 properties.csv
-rw-r--r-- 1 nholtz nholtz   31 Jun  3 18:10 releases.csv
-rw-r--r-- 1 nholtz nholtz  936 Jun  3 18:10 signatures.csv
-rw-r--r-- 1 nholtz nholtz   38 Jun  3 18:10 support_displacements.csv
-rw-r--r-- 1 nholtz nholtz   36 Jun  3 18:10 supports.csv


In [46]:
##test:
%cat test-1.d/signatures.csv

NAME,PATH,SIGNATURE
member_loads,test-1.d/member_loads.csv,dd250084236a127fdd54f5003caba9d7cb980efae8bec6405ba84d899887e43f
releases,test-1.d/releases.csv,ae3d4681f60b6c5665cfd07a4894a01dd386cc7a51e98eb08248fc202e41e09f
supports,test-1.d/supports.csv,c9b14f3fb0868a679912d6d5d8f23e75f89cc8accd8e9f2b1613bc9710653815
support_displacements,test-1.d/support_displacements.csv,b475cca77a5ddcf815425514c484fcb3506195db840a9bf3006a187c9e457617
load_combinations,test-1.d/load_combinations.csv,efe64703cadacf5f169f913b9d49dcf4c6523dd62e474cfff90782787c830310
members,test-1.d/members.csv,783cee2e924a1f1fc738638af431671f02e25f1b97430d834a226eecd3dd09e3
nodes,test-1.d/nodes.csv,a1c6202ec4a3f745cbde4e96c7b07ce173d14ebaeaa6956b40260053ffb34e6c
node_loads,test-1.d/node_loads.csv,c6283d540ec54fe3f9a92ac7f0ec98e74f3e8351dc77387e3d113bc5e618ced4
properties,test-1.d/properties.csv,4ebc2477cadb207459034659d0bf1da9b9dbb02bb06af2c060894ecb595effca


## Results Output

In [47]:
##test:
import Frame2D_SolveFirstOrder
f.reset()
f.input_all()
rs = f.solve('all')

### Node Displacements

In [48]:
@extend
class Frame2D:
    
    COLUMNS_node_displacements = ['NODEID','DX','DY','RZ']
    
    def list_node_displacements(self,rs):
        if not hasattr(rs,'node_displacements'):
            return []
        ans = []
        D = rs.node_displacements
        for node in self.nodes.values():
            d = D[node.dofnums]
            ans.append((node.id,d[0,0],d[1,0],d[2,0]))
        return ans

In [49]:
##test:
f.list_node_displacements(rs)

[('A', 0.0, -10.0, 0.0),
 ('B', -96.09609609609663, -10.447154471544724, -0.016733181727309074),
 ('C', -96.09609609609663, -0.52845528455284552, 0.020493594228975075),
 ('D', 0.0, 0.0, 0.024024024024024156)]

In [50]:
##test:
f.write_table('node_displacements',ds_name='test-1',prefix=rs.loadcase,record=False,precision=15,args=(rs,))
%cat test-1.d/all/node_displacements.csv

NODEID,DX,DY,RZ
A,0,-10,0
B,-96.0960960960966,-10.4471544715447,-0.0167331817273091
C,-96.0960960960966,-0.528455284552846,0.0204935942289751
D,0,0,0.0240240240240242


### Reaction Forces

In [51]:
@extend
class Frame2D:
    
    COLUMNS_reaction_forces = ['NODEID','FX','FY','MZ']
    
    def list_reaction_forces(self,rs):
        if not hasattr(rs,'reaction_forces'):
            return []
        R = rs.reaction_forces
        ans = []
        for node in self.nodes.values():
            if node.constraints:
                l = [node.id,None,None,None]
                for dirn in node.constraints:
                    i = node.DIRECTIONS[dirn]
                    j = node.dofnums[i]
                    val = R[j-self.nfree,0]
                    l[i+1] = val
                ans.append(l)
        return ans

In [52]:
##test:
f.list_reaction_forces(rs)

[['A', 200000.00000000111, 275000.00000000559, -800000000.00000441],
 ['D', 0.0, 325000.0, None]]

In [53]:
##test:
f.write_table('reaction_forces',ds_name='test-1',prefix=rs.loadcase,record=False,precision=15,args=(rs,))
%cat test-1.d/all/reaction_forces.csv

NODEID,FX,FY,FZ
A,200000.000000001,275000.000000006,-800000000.000004
D,0,325000,


### Member End Forces

In [54]:
@extend
class Frame2D:
    
    COLUMNS_member_end_forces = 'MEMBERID,FXJ,FYJ,MZJ,FXK,FYK,MZK'.split(',')
    
    def list_member_end_forces(self,rs):
        if not hasattr(rs,'member_efs'):
            return []
        mefs = rs.member_efs
        ans = []
        for memb in self.members.values():
            efs = mefs[memb].fefs
            ans.append((memb.id,efs[0,0],efs[1,0],efs[2,0],efs[3,0],efs[4,0],efs[5,0]))
        return ans

In [55]:
##test:
f.list_member_end_forces(rs)

[('AB',
  275000.00000000559,
  -200000.00000000111,
  -800000000.00000441,
  -275000.00000000559,
  200000.00000000111,
  0.0),
 ('BC', 0.0, 275000.0, 0.0, 0.0, 325000.0, -5.9604644775390625e-08),
 ('CD', 325000.0, 0.0, 0.0, -325000.0, 0.0, 0.0)]

In [56]:
##test:
f.write_table('member_end_forces',ds_name='test-1',prefix=rs.loadcase,record=False,precision=15,args=(rs,))
%cat test-1.d/all/member_end_forces.csv

MEMBERID,FXJ,FYJ,MZJ,FXK,FYK,MZK
AB,275000.000000006,-200000.000000001,-800000000.000004,-275000.000000006,200000.000000001,0
BC,0,275000,0,0,325000,-5.96046447753906e-08
CD,325000,0,0,-325000,0,0


### $P-\Delta$ Forces
See module ```..._Display```

### All Results

In [57]:
@extend
class Frame2D:
    
    def write_results(self,ds_name,rs):
        self.write_table('node_displacements',ds_name=ds_name,prefix=rs.loadcase,record=False,
                         precision=15,args=(rs,),makedir=True)
        self.write_table('reaction_forces',ds_name=ds_name,prefix=rs.loadcase,record=False,
                         precision=15,args=(rs,))
        self.write_table('member_end_forces',ds_name=ds_name,prefix=rs.loadcase,record=False,
                         precision=15,args=(rs,))
        if rs.pdelta:
            self.write_table('pdelta_forces',ds_name=ds_name,prefix=rs.loadcase,record=False,
                             precision=15,args=(rs,))


In [58]:
##test:
f.reset()
f.input_all()
rs = f.solve('One')
f.write_results('test-1',rs)

%ls -l test-1.d/One
%cat test-1.d/One/node_displacements.csv
%cat test-1.d/One/reaction_forces.csv
%cat test-1.d/One/member_end_forces.csv

total 12
-rw-r--r-- 1 nholtz nholtz 256 Jun  3 18:10 member_end_forces.csv
-rw-r--r-- 1 nholtz nholtz 166 Jun  3 18:10 node_displacements.csv
-rw-r--r-- 1 nholtz nholtz 102 Jun  3 18:10 reaction_forces.csv
NODEID,DX,DY,RZ
A,0,0,0
B,-168.168168168169,-0.670731707317113,-0.0269747725909636
C,-168.168168168169,-0.792682926829268,0.0288653913434626
D,0,0,0.0420420420420423
NODEID,FX,FY,FZ
A,350000.000000002,412500.000000024,-1400000000.00001
D,-5.82076609134674e-11,487500,
MEMBERID,FXJ,FYJ,MZJ,FXK,FYK,MZK
AB,412500.000000024,-350000.000000002,-1400000000.00001,-412500.000000024,350000.000000002,0
BC,0,412500,-1.19209289550781e-07,0,487500,-1.19209289550781e-07
CD,487500,5.82076609134674e-11,0,-487500,-5.82076609134674e-11,0
